# Food Bank Problem

In [2]:
import sys
import numpy as np
import plotly.express as px
import pandas as pd
import scipy.optimize as optimization

## OPT - Waterfilling

In [3]:
## Water-filling Algorithm for sorted demands
def waterfilling_sorted(d,b):
    n = np.size(d)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        equal_allocation = bundle_remaining/(n-i)
        if d[i]<equal_allocation:
            allocations[i] = bundle_remaining if i==n-1 else d[i]
        else:
            allocations[i] = equal_allocation
        bundle_remaining -= allocations[i]
    return allocations

In [60]:
## Water-filling Algorithm for sorted demands and weights for bucket width
def waterfilling_sorted_weights(demands, weights,budget,index, width):
    #print(demands, weights,budget,index, width)
    n = np.size(demands)
    allocations = np.zeros(n)
    budget_remaining = budget
    weight_index = 0
    equal_allocation = budget_remaining/(width+1)
    index_found = False
    curr_weight = 1
    for i in range(n):
        if demands[i]<equal_allocation:
            allocations[i] = budget_remaining if i==n-1 else demands[i]
        else:
            allocations[i] = equal_allocation
        if i!=index:
            budget_remaining -= allocations[i]*weights[weight_index]*width
            curr_weight -= weights[weight_index]   
            weight_index+=1
        else:
            budget_remaining -= allocations[i]
            index_found = True
        if i!=n-1:
            if index_found:
                equal_allocation = budget_remaining/(width*curr_weight)
            else: 
                equal_allocation = budget_remaining/(width*curr_weight+1)
    return allocations

In [61]:
waterfilling_weights(np.array([0.5,0.5]), np.array([1,2]),np.array([2,1]), 3)

array([1.66666667, 1.33333333])

In [5]:
## Water-filling Algorithm for general demands
def waterfilling(d,b):
    n = np.size(d)
    sorted_indices = np.argsort(d)
    sorted_demands = np.sort(d)
    sorted_allocations = waterfilling_sorted(sorted_demands, b)
    allocations = np.zeros(n)
    for i in range(n):
        allocations[sorted_indices[i]] = sorted_allocations[i]
    return allocations

In [6]:
## Tests
assert list(waterfilling(np.zeros(0), 5)) == []
assert list(waterfilling(np.array([1,2,3,4]), 10)) == [1,2,3,4]
assert list(waterfilling(np.array([3,4,1,2]), 10)) == [3,4,1,2]
assert list(waterfilling(np.array([1,2,3,4]), 8)) == [1,2,2.5,2.5]
assert list(waterfilling(np.array([3,1,4,2]), 8)) == [2.5,1,2.5,2]
assert list(waterfilling(np.array([3,6,5,6]), 8)) == [2,2,2,2]

## Online Algorithms

In [7]:
## Online Water-filling taking minimum of realized demand and B/n
def waterfilling_proportional(demands_realized,b):
    n = np.size(demands_realized)
    allocations = np.zeros(n)
    eq = 1 if n==0 else b/n
    bundle_remaining = b
    for i in range(n):
        if i!=n-1:
            allocations[i] = min(eq, demands_realized[i])
        else:
            allocations[i] = bundle_remaining
        bundle_remaining -= allocations[i]
    return allocations

In [8]:
## Online Water-filling taking minimum of realized demand and B/n
def waterfilling_proportional_remaining(demands_realized,b):
    n = np.size(demands_realized)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        if i!=n-1:
            allocations[i] = min(bundle_remaining/(n-i), demands_realized[i])
        else:
            allocations[i] = bundle_remaining
        bundle_remaining -= allocations[i]
    return allocations

In [9]:
## Tests
assert list(waterfilling_proportional(np.zeros(0), 5)) == []
assert list(waterfilling_proportional(np.array([1,2,3,4]), 10)) == [1,2,2.5,4.5]
assert list(waterfilling_proportional(np.array([3,4,1,2]), 10)) == [2.5,2.5,1,4]
assert list(waterfilling_proportional(np.array([1,2,3,4]), 8)) == [1,2,2,3]
assert list(waterfilling_proportional(np.array([3,1,4,2]), 8)) == [2,1,2,3]
assert list(waterfilling_proportional(np.array([3,6,5,6]), 8)) == [2,2,2,2]

In [10]:
## Tests
assert list(waterfilling_proportional_remaining(np.zeros(0), 5)) == []
assert list(waterfilling_proportional_remaining(np.array([1,2,3,4]), 10)) == [1,2,3,4]
assert list(waterfilling_proportional_remaining(np.array([3,4,1,2]), 10)) == [2.5,2.5,1,4]
assert list(waterfilling_proportional_remaining(np.array([1,2,3,4]), 8)) == [1,2,2.5,2.5]
assert list(waterfilling_proportional_remaining(np.array([3,1,4,2]), 8)) == [2,1,2.5,2.5]
assert list(waterfilling_proportional_remaining(np.array([3,6,5,6]), 8)) == [2,2,2,2]
assert list(waterfilling_proportional_remaining(np.array([4,5,3,6]), 16)) == [4,4,3,5]
assert list(waterfilling_proportional_remaining(np.array([9,10,2,1]), 16)) == [4,4,2,6]

In [11]:
## Online Water-filling taking minimum of realized demand and predetermeined allocation
def waterfilling_online_1(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    prior_allocations_assignment = waterfilling(demands_predicted,b)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        allocations[i] = min(prior_allocations_assignment[i], demands_realized[i]) if i!=n-1 else bundle_remaining
        bundle_remaining -= allocations[i]
    return allocations

In [12]:
## Tests
assert list(waterfilling_online_1(np.zeros(0), np.zeros(0), 5)) == []
assert list(waterfilling_online_1(np.array([1,2,3,4]), np.array([5,5,5,5]), 10)) == [1,2,3,4]
assert list(waterfilling_online_1(np.array([3,1,4,2]), np.array([2,3,2.5,1]), 8)) == [2,1,2.5,2.5]


In [13]:
## Online Water-filling algorithm where each agent solves waterfilling with realized current demand and expected following demands
def waterfilling_online_2(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        allocations[i] = waterfilling(np.append(demands_realized[i],demands_predicted[i+1:]), bundle_remaining)[0]
        bundle_remaining -= allocations[i]
    return allocations

In [26]:
def insert_sorted(lst, element):
    n = np.size(lst)
    if n==0:
        return np.array([element]),0
    if element<=lst[0]:
        return np.append(element,lst),0
    if element>=lst[n-1]:
        return np.append(lst,element),n
    left = 0
    right = n-1
    while left<right-1:
        mid_ind = int((left+right)/2)
        if element<lst[mid_ind]:
            right = mid_ind
        elif element > lst[mid_ind] :
            left = mid_ind
        if element == lst[mid_ind] or (element>lst[mid_ind] and element<lst[mid_ind+1]):
            return np.append(np.append(lst[:mid_ind+1],element),lst[mid_ind+1:]), mid_ind+1
    return np.append(np.append(lst[:left],element),lst[left:]), left

In [15]:
def delete_sorted(lst,element):
    n = np.size(lst)
    if element==lst[0]:
        return lst[1:]
    if element==lst[n-1]:
        return lst[:-1]
    left = 0
    right = n-1
    while left<right-1:
        mid_ind = int((left+right)/2)
        if element<lst[mid_ind]:
            right = mid_ind
        elif element > lst[mid_ind] :
            left = mid_ind
        else:
            return np.append(lst[:mid_ind],lst[mid_ind+1:])

In [16]:
## O(n^2) version of online algorithm that needs waterfilling evaluated multiple times
def waterfilling_dynamic(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    sorted_demands = np.sort(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        sorted_demands = delete_sorted(sorted_demands, demands_predicted[i])
        new_sorted_list,index = insert_sorted(sorted_demands,demands_realized[i])
        if i<n-1:
            allocations[i] = min((waterfilling_sorted(new_sorted_list, bundle_remaining))[index],demands_realized[i])
        else:
            allocations[i] = bundle_remaining
        bundle_remaining -= allocations[i]
    return allocations
    

In [16]:
## O(n^2) version of online algorithm that needs waterfilling evaluated multiple times and budget over-estimated at first
def waterfilling_dynamic_budget_opt(demands_predicted, demands_realized, b, factor):
    n = np.size(demands_predicted)
    sorted_demands = np.sort(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        sorted_demands = delete_sorted(sorted_demands, demands_predicted[i])
        new_sorted_list,index = insert_sorted(sorted_demands,demands_realized[i])
        if i<n-1:
            potential_alloc = min((waterfilling_sorted(new_sorted_list, bundle_remaining+factor*(n-i-1)/n))[index],demands_realized[i])
            if potential_alloc>=bundle_remaining:
                allocations[i] = min((waterfilling_sorted(new_sorted_list, bundle_remaining))[index],demands_realized[i])
            else:
                allocations[i] = potential_alloc
        else:
            allocations[i] = bundle_remaining
        bundle_remaining -= allocations[i]
    return allocations
    

In [16]:
## O(n^2) version of online algorithm that needs waterfilling evaluated multiple times and budget under-estimated at first
def waterfilling_dynamic_budget_pess(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    sorted_demands = np.sort(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        sorted_demands = delete_sorted(sorted_demands, demands_predicted[i])
        new_sorted_list,index = insert_sorted(sorted_demands,demands_realized[i])
        if i<n-1:
            allocations[i] = min((waterfilling_sorted(new_sorted_list, bundle_remaining-10*(n-i)/n))[index],demands_realized[i])
        else:
            allocations[i] = bundle_remaining
        bundle_remaining -= allocations[i]
    return allocations
    

In [1]:
## budget underestimated at first, and overestimated at end.
def waterfilling_weights_budget_adjust(weights, sorted_distribution, demands_realized, budget,factor_pess, factor_opt, turn):
    n = np.size(demands_realized)
    distribution_size = np.size(sorted_distribution)
    distribution_weighted = weights*sorted_distribution
    allocations = np.zeros(n)
    budget_remaining = budget
    turning_point = int(turn*n)
    for i in range(turning_point):
        new_sorted_list,index = insert_sorted(sorted_distribution,demands_realized[i])
        if i<n-1 :
            if factor*(n-i-1)/n<budget_remaining:
                allocations[i] = min((waterfilling_sorted_weights(new_sorted_list, weights,budget_remaining-factor_pess*(n-i-1)/n,index,n-i-1))[index],demands_realized[i])
            else:
                allocations[i] = min((waterfilling_sorted_weights(new_sorted_list, weights,budget_remaining,index,n-i-1))[index],demands_realized[i])
        else:
            allocations[i] = budget_remaining
        budget_remaining -= allocations[i]
        
    for i in range(turning_point,n):
        new_sorted_list,index = insert_sorted(sorted_distribution,demands_realized[i])
        if i<n-1 :
            potential_alloc = min((waterfilling_sorted_weights(new_sorted_list, weights,budget_remaining+factor_opt*(n-i-1)/n,index,n-i-1))[index],demands_realized[i])
            if potential_alloc>=budget_remaining:
                allocations[i] = min((waterfilling_sorted(new_sorted_list, budget_remaining))[index],demands_realized[i])
            else:
                allocations[i] = potential_alloc
        else:
            allocations[i] = budget_remaining
        budget_remaining -= allocations[i]
    return allocations

In [17]:
## Tests 
assert list(waterfilling_online_2(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_online_2(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_online_2(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_online_2(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]

assert list(waterfilling_dynamic(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_dynamic(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_dynamic(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_dynamic(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]
assert list(waterfilling_dynamic(np.array([4,5,3,6]), np.array([9,10,2,1]), 30)) == [9,10,2,9]

In [62]:
## Waterfilling using weighted bars
def waterfilling_weights(weights, sorted_distribution, demands_realized, budget):
    n = np.size(demands_realized)
    distribution_size = np.size(sorted_distribution)
    distribution_weighted = weights*sorted_distribution
    allocations = np.zeros(n)
    budget_remaining = budget
    for i in range(n):
        new_sorted_list,index = insert_sorted(sorted_distribution,demands_realized[i])
        if i<n-1 :
            allocations[i] = min((waterfilling_sorted_weights(new_sorted_list, weights,budget_remaining,index,n-i-1))[index],demands_realized[i])
        else:
            allocations[i] = budget_remaining
        budget_remaining -= allocations[i]
    return allocations
    

In [84]:
## Tests 
assert list(waterfilling_weights(np.array([0.5,0.5]), np.array([1,2]),np.zeros(0), 0)) == []
assert list(np.around(waterfilling_weights(np.array([0.5,0.5]), np.array([1,2]), np.array([2,1]), 3),2)) == [1.67,1.33]
assert list(waterfilling_weights(np.array([0.5,0.5]), np.array([1,2]),np.array([2,1,2]), 5)) == [2,1,2]
assert list(waterfilling_weights(np.array([0.5,0.5]), np.array([1,2]),np.array([2,1,1]), 4)) == [1.5,1,1.5]
assert list(waterfilling_weights(np.array([0.5,0.5]), np.array([1,2]),np.array([2,1,2,1,2]), 8)) == [2,1,2,1,2]
assert list(np.around(waterfilling_weights(np.array([0.5,0.5]), np.array([1,2]),np.array([2,2,1,1,2]), 8),1)) == [2,1.8,1,1,2.2]
assert list(np.around(waterfilling_weights(np.array([0.5,0.5]), np.array([1,2]),np.array([2,2,1,1,2]), 6),1)) == [1.3,1.3,1,1,1.4]
assert list(np.around(waterfilling_weights(np.array([0.5,0.5]), np.array([1,2]),np.array([2,2,1,1,2]), 10),2)) == [2,2,1,1,4]


In [ ]:
## Dynamic waterfilling algorithm that is more optimistic about town it is currently visiting than future towns
def waterfilling_optimistic(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    sorted_demands = np.sort(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        sorted_demands = delete_sorted(sorted_demands-(n-i-1)/np.sqrt(n), demands_predicted[i])
        new_sorted_list,index = insert_sorted(sorted_demands,demands_realized[i])
        allocations[i] = (waterfilling_sorted(new_sorted_list, bundle_remaining))[index]
        bundle_remaining -= allocations[i]
    return allocations

In [ ]:
## Dynamic waterfilling algorithm that is more pessimistic about town it is currently visiting than future towns
def waterfilling_pessimistic(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    sorted_demands = np.sort(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        sorted_demands = delete_sorted(sorted_demands+(n-i-1)/np.sqrt(n), demands_predicted[i])
        new_sorted_list,index = insert_sorted(sorted_demands,demands_realized[i])
        allocations[i] = (waterfilling_sorted(new_sorted_list, bundle_remaining))[index]
        bundle_remaining -= allocations[i]
    return allocations

In [12]:
def waterfilling_online_3(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        future_allocations = waterfilling(np.append(np.Inf,demands_predicted[i+1:]), bundle_remaining)
        if future_allocations[0]>demands_realized[i] and i!=n-1:
            allocations[i] = demands_realized[i]
        else:
            allocations[i] = future_allocations[0]
        bundle_remaining -= allocations[i]
    return allocations

In [13]:
## Online Water-filling algorithm where each agent is assigned infinite demand while finding optimal solution and allocation is readjusted
def waterfilling_online_3(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        future_allocations = waterfilling(np.append(np.Inf,demands_predicted[i+1:]), bundle_remaining)
        if future_allocations[0]>demands_realized[i] and i!=n-1:
            allocations[i] = demands_realized[i]
        else:
            allocations[i] = future_allocations[0]
        bundle_remaining -= allocations[i]
    return allocations

In [14]:
## Tests 
assert list(waterfilling_online_3(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_online_3(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_online_3(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_online_3(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]

3.0
2.666666666666667
2.6666666666666665
2.6666666666666665
2.0
1.0
6.0
6.0
4.0
4.0
2.0
5.0


## Objective Functions

In [15]:
## Calculate log of Nash welfare for objective function
def objective_nash_log(demands, allocation):
    welfare_sum = 0
    for i in range(np.size(demands)):
        welfare_sum += np.log(min(1,allocation[i]/demands[i]))
    return welfare_sum

In [ ]:
## Calculate log of Nash welfare for objective function
def objective_nash(demands, allocation):
    welfare_product = 1
    for i in range(np.size(demands)):
        welfare_product = welfare_product*min(1,allocation[i]/demands[i])
    return welfare_sum

## Experiment

In [16]:
def make_demands_uniform_distribution(num_towns, demand_ranges):
    demands = np.zeros(num_towns)
    expected_demands = np.zeros(num_towns)
    for i in range(num_towns):
        demands[i] = np.random.uniform(0, demand_ranges[i])
        expected_demands[i] = demand_ranges[i]/2
    return demands, expected_demands


In [17]:
def make_demands_exponential_distribution(num_towns, demand_means):
    demands = np.zeros(num_towns)
    expected_demands = np.zeros(num_towns)
    for i in range(num_towns):
        demands[i] = np.random.exponential(demand_means[i])
        expected_demands[i] = demand_means[i]
    return demands, expected_demands
